In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [ ]:
df = pd.read_csv('data/US_Accidents_March23.csv')

    ID   Source  Severity           Start_Time             End_Time  \
0  A-1  Source2         3  2016-02-08 05:46:00  2016-02-08 11:00:00   
1  A-2  Source2         2  2016-02-08 06:07:59  2016-02-08 06:37:59   
2  A-3  Source2         2  2016-02-08 06:49:27  2016-02-08 07:19:27   
3  A-4  Source2         3  2016-02-08 07:23:34  2016-02-08 07:53:34   
4  A-5  Source2         2  2016-02-08 07:39:07  2016-02-08 08:09:07   

   Start_Lat  Start_Lng  End_Lat  End_Lng  Distance(mi)  ... Roundabout  \
0  39.865147 -84.058723      NaN      NaN          0.01  ...      False   
1  39.928059 -82.831184      NaN      NaN          0.01  ...      False   
2  39.063148 -84.032608      NaN      NaN          0.01  ...      False   
3  39.747753 -84.205582      NaN      NaN          0.01  ...      False   
4  39.627781 -84.188354      NaN      NaN          0.01  ...      False   

  Station   Stop Traffic_Calming Traffic_Signal Turning_Loop Sunrise_Sunset  \
0   False  False           False          F

In [3]:
most_common_state = df["State"].value_counts().head(5)
entry_count = df["State"].value_counts().max()
print(f"The most common state is {most_common_state} with {entry_count} entries.")
print("Procent of the total data used: ", (entry_count / len(df)) * 100)

The most common state is State
CA    1741433
FL     880192
TX     582837
SC     382557
NY     347960
Name: count, dtype: int64 with 1741433 entries.
Procent of the total data used:  22.532922105161823


In [4]:
ca_df = df[df["State"] == "CA"]
CA_LA_df = ca_df[ca_df["City"] == "Los Angeles"]

We can use the start latitude and longittude for the project, no need for end latitude and longitude so no removing of missing data at this point.

In [5]:
# Save to a new CSV file
CA_LA_df.to_csv("data/CA_LA_subset.csv", index=False)

All generated datasets are saved under /data/ folder, used for further analysis and implementation later.

In [ ]:
import pandas as pd
df = pd.read_csv('data/CA_LA_subset.csv')
print(df.head())

CA_LA_df = df[["Start_Time", "Start_Lat", "Start_Lng","Visibility(mi)"]]
print(CA_LA_df.head())

# Save to a new CSV file
CA_LA_df.to_csv("data/LA_df_time_coords.csv", index=False)

        ID   Source  Severity           Start_Time             End_Time  \
0  A-42867  Source2         2  2016-06-21 10:46:30  2016-06-21 11:27:00   
1  A-42868  Source2         3  2016-06-21 10:49:21  2016-06-21 11:34:21   
2  A-42882  Source2         3  2016-06-21 10:51:45  2016-06-21 11:36:45   
3  A-42884  Source2         3  2016-06-21 10:56:24  2016-06-21 11:34:00   
4  A-42899  Source2         3  2016-06-21 11:30:46  2016-06-21 12:00:46   

   Start_Lat   Start_Lng  End_Lat  End_Lng  Distance(mi)  ... Roundabout  \
0  34.078926 -118.289040      NaN      NaN           0.0  ...      False   
1  34.091179 -118.239471      NaN      NaN           0.0  ...      False   
2  34.037239 -118.309074      NaN      NaN           0.0  ...      False   
3  34.027458 -118.274490      NaN      NaN           0.0  ...      False   
4  33.947544 -118.279434      NaN      NaN           0.0  ...      False   

  Station   Stop Traffic_Calming Traffic_Signal Turning_Loop Sunrise_Sunset  \
0   False  Fa

Weather data processing

In [ ]:
data = pd.read_csv('data/LA_df_time_coords.csv')
data

In [ ]:
# Convert the 'Start_Time' column to datetime format, handling mixed formats while keeping 'Start Lat' and 'Start Lng' as strings

data['Start_Time'] = pd.to_datetime(data['Start_Time'], format='mixed')
# Extract the hour from the 'Start_Time' column
data['Start_hour'] = data['Start_Time'].dt.hour
# Create a new DataFrame which contains the unique starts of hours and the date
# Extract the date from the 'Start_Time' column
data['Start_date'] = data['Start_Time'].dt.date
# Create a new DataFrame with unique combinations of date and hour
data['Start_hour'] = data['Start_hour'].astype(int)
# Drop duplicates to get unique combinations of date and hour
unique_starts = data[['Start_date', 'Start_hour','Start_Lat','Start_Lng','Visibility(mi)']].drop_duplicates()
unique_starts.index = range(len(unique_starts))
#maximum date
max_date = unique_starts['Start_date'].max()
max_hour = unique_starts['Start_hour'].max()
min_date = unique_starts['Start_date'].min()
min_hour = unique_starts['Start_hour'].min()
print(f"Max date: {max_date}, Max hour: {max_hour}")
print(f"Min date: {min_date}, Min hour: {min_hour}")
unique_starts


In [ ]:
weather_data = pd.read_csv('data/weather_data.csv')
# convert the 'datetime' column to two columns: 'date' and 'hour'
weather_data['date'] = pd.to_datetime(weather_data['datetime']).dt.date
weather_data['hour'] = pd.to_datetime(weather_data['datetime']).dt.hour
# Drop the original 'datetime' column
weather_data = weather_data.drop(columns=['datetime'])
weather_data

In [ ]:
# Connect the two dataframes on the date and hour columns only if the date exists in the unique_starts
# Merge the unique_starts DataFrame with the weather_data DataFrame
merged_data = pd.merge(unique_starts, weather_data, left_on=['Start_date', 'Start_hour'], right_on=['date', 'hour'], how='left')
merged_data.drop(columns=['date', 'hour'], inplace=True)
merged_data.to_csv('data/merged_data.csv', index=False)
merged_data